This notebook is based on the book Advanced Analytics with Spark by Sandy Ryza, Uri Laserson, Sean Owen, and Josh Wills.

Data https://bit.ly/1Aoywaq

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`
import org.apache.spark.sql._

import $ivy.$                                  

import org.apache.spark.sql._

In [2]:
import org.apache.log4j.{Level,Logger}
Logger.getLogger("org").setLevel(Level.ERROR)

import org.apache.log4j.{Level,Logger}


## Create the sparkSession

In [3]:
val spark = {
            NotebookSparkSession.builder()
            .master("local[*]")
            .getOrCreate()
            }

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@3ec0e10b

## GetData and parse it 

    Note that in this case the schema is being infered. The caused some problems later when converting to a DS as there was not Encoder available for the tuples within the Row object of the dataframe. So, I used a case class and the as method to get the desired schema for the dataset.

In [4]:
val parsedData = spark.read
                    .option("header","true")
                    .option("inferSchema","true")
                    .option("nullValue","?")
                    .csv("../linkages/")

csv at cmd3.sc:5

1 / 1

csv at cmd3.sc:5

10 / 10

parsedData: DataFrame = [id_1: int, id_2: int ... 10 more fields]

In [5]:
parsedData.take(10)

take at cmd4.sc:1

1 / 1

res4: Array[Row] = Array(
  [3148,8326,1.0,null,1.0,null,1,1,1,1,1,true],
  [14055,94934,1.0,null,1.0,null,1,1,1,1,1,true],
  [33948,34740,1.0,null,1.0,null,1,1,1,1,1,true],
  [946,71870,1.0,null,1.0,null,1,1,1,1,1,true],
  [64880,71676,1.0,null,1.0,null,1,1,1,1,1,true],
  [25739,45991,1.0,null,1.0,null,1,1,1,1,1,true],
  [62415,93584,1.0,null,1.0,null,1,1,1,1,0,true],
  [27995,31399,1.0,null,1.0,null,1,1,1,1,1,true],
  [4909,12238,1.0,null,1.0,null,1,1,1,1,1,true],
  [15161,16743,1.0,null,1.0,null,1,1,1,1,1,true]
)

In [6]:
parsedData.printSchema

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



Note that the Id's are nullable even though they are the primary keys in this dataset. This was a work around as there appeared to be a null row on one occassion, possibly due to hidden file being ingested in the read.

In [7]:
parsedData.cache()

res6: DataFrame = [id_1: int, id_2: int ... 10 more fields]

In [8]:
parsedData.count()

count at cmd7.sc:1

10 / 10

count at cmd7.sc:1

1 / 1

res7: Long = 5749132L

## Basic exploration

In [9]:
parsedData.groupBy("is_match").count().show()

show at cmd8.sc:1

10 / 10

show at cmd8.sc:1

1 / 1

show at cmd8.sc:1

...

show at cmd8.sc:1

100 / 100

show at cmd8.sc:1

75 / 75

+--------+-------+
|is_match|  count|
+--------+-------+
|    true|  20931|
|   false|5728201|
+--------+-------+



In [10]:
parsedData.where("is_match=null").show()

+----+----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
|id_1|id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+----+----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
+----+----+------------+------------+------------+------------+-------+------+------+------+-------+--------+



implicits makes contains a number of methods. The one used here is the column object creation $"count". This creates the column object which as a desc property (description NOT descending!).

In [11]:
import spark.implicits._
import org.apache.spark.sql.functions._

import spark.implicits._

import org.apache.spark.sql.functions._

In [12]:
parsedData.groupBy("is_match").count().orderBy($"count".desc).show()

show at cmd11.sc:1

10 / 10

show at cmd11.sc:1

200 / 200

+--------+-------+
|is_match|  count|
+--------+-------+
|   false|5728201|
|    true|  20931|
+--------+-------+



In [13]:
parsedData.agg(avg($"cmp_sex"),stddev($"cmp_sex")).show()

show at cmd12.sc:1

10 / 10

show at cmd12.sc:1

1 / 1

+-----------------+--------------------+
|     avg(cmp_sex)|stddev_samp(cmp_sex)|
+-----------------+--------------------+
|0.955001381078048| 0.20730111116897781|
+-----------------+--------------------+



## Create a case class to use as a schema when converting the dataframe to dataset. 

org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this) This line prevents and error related to the scope of the case classs. Inner vs outer see https://stackoverflow.com/questions/40229953/spark-dataset-example-unable-to-generate-an-encoder-issue
Note: when using this in the as function on the dataframe using as[MatchClass] fails because the conversion to the javatypes to the spark types requires an explict encoder (required of all non-primatives?). To make the conversion work we create an org.apache.spark.sql.Encoder.product[MatchClass] object to use instead of the MatchClass directly.

In [14]:
org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)

case class MatchData(
id_1: Option[Int],
id_2: Option[Int],
cmp_fname_c1: Option[Double],
cmp_fname_c2: Option[Double],
cmp_lname_c1: Option[Double],
cmp_lname_c2: Option[Double],
cmp_sex: Option[Int],
cmp_bd: Option[Int],
cmp_bm: Option[Int],
cmp_by: Option[Int],
cmp_plz: Option[Int],
is_match: Option[Boolean]
)

defined class MatchData

In [15]:
parsedData.take(10)

take at cmd14.sc:1

1 / 1

res14: Array[Row] = Array(
  [3148,8326,1.0,null,1.0,null,1,1,1,1,1,true],
  [14055,94934,1.0,null,1.0,null,1,1,1,1,1,true],
  [33948,34740,1.0,null,1.0,null,1,1,1,1,1,true],
  [946,71870,1.0,null,1.0,null,1,1,1,1,1,true],
  [64880,71676,1.0,null,1.0,null,1,1,1,1,1,true],
  [25739,45991,1.0,null,1.0,null,1,1,1,1,1,true],
  [62415,93584,1.0,null,1.0,null,1,1,1,1,0,true],
  [27995,31399,1.0,null,1.0,null,1,1,1,1,1,true],
  [4909,12238,1.0,null,1.0,null,1,1,1,1,1,true],
  [15161,16743,1.0,null,1.0,null,1,1,1,1,1,true]
)

In [16]:
parsedData.printSchema

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [17]:
val caseClassschema = Encoders.product[MatchData].schema

caseClassschema: types.StructType = StructType(
  StructField("id_1", IntegerType, true, {}),
  StructField("id_2", IntegerType, true, {}),
  StructField("cmp_fname_c1", DoubleType, true, {}),
  StructField("cmp_fname_c2", DoubleType, true, {}),
  StructField("cmp_lname_c1", DoubleType, true, {}),
  StructField("cmp_lname_c2", DoubleType, true, {}),
  StructField("cmp_sex", IntegerType, true, {}),
  StructField("cmp_bd", IntegerType, true, {}),
  StructField("cmp_bm", IntegerType, true, {}),
  StructField("cmp_by", IntegerType, true, {}),
  StructField("cmp_plz", IntegerType, true, {}),
  StructField("is_match", BooleanType, true, {})
)

Here is the csv read again without relying on schema inference. Its is given explicitly via the schema function which takes the schema of the Encoder.product[MatchClass] (think of product as cartesian product of types).

In [18]:
val parsedData = spark.read.schema(caseClassschema)
                    .option("header","true")
                    .option("nullValue","?")
                    .csv("../linkages/")              

parsedData: DataFrame = [id_1: int, id_2: int ... 10 more fields]

To get the below code to work I had to modify the case class definition to make the id's and the is_match column  nullable i.e. wrap them in Option[]. There appears to be an empty row that is causingthe problem. Further investigation neeeded.

In [19]:
val scData = parsedData.as[MatchData]

scData: Dataset[MatchData] = [id_1: int, id_2: int ... 10 more fields]

In [20]:
scData.show()

show at cmd19.sc:1

1 / 1

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| 3148| 8326|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|14055|94934|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|33948|34740|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|  946|71870|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|64880|71676|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|25739|45991|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|  

In [21]:
scData.printSchema

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



Simple methods for scoring the match data which contains binary indicators of some algorithm (not specific here) determined that two fields are to be considered the same. The score is the sum of the matches with missing data skipped.

In [22]:
/**
Create a case class Score and define addition. Note that the value parameter is exported because this is a case class.
@param value initialisation parameter for case class creation
**/
case class Score(value: Double){
    //Adds the right hand value oi to the initialisation value an
    //returns: Score object.
    def +(oi: Option[Int])={
        Score(value+oi.getOrElse(0))
    }
}
// teh score just represents the sum of the match flags==1.
def scoreMatchData(md: MatchData): Double = {
    (Score(md.cmp_lname_c1.getOrElse(0.0))+md.cmp_plz+md.cmp_by+md.cmp_bd+md.cmp_bm).value
}

defined class Score
defined function scoreMatchData

In [23]:
val scored = scData.map(x=>(scoreMatchData(x),x.is_match)).toDF("score","is_match")

scored: DataFrame = [score: double, is_match: boolean]

In [24]:
scored.cache()

res23: DataFrame = [score: double, is_match: boolean]

In [25]:
scored.take(10)

take at cmd24.sc:1

1 / 1

res24: Array[Row] = Array(
  [5.0,true],
  [5.0,true],
  [5.0,true],
  [5.0,true],
  [5.0,true],
  [5.0,true],
  [4.0,true],
  [5.0,true],
  [5.0,true],
  [5.0,true]
)

In [26]:
def crossTabs(scored: DataFrame, t:Double): DataFrame = {
    scored.
        selectExpr(s"score>=$t as Above","is_match").
        groupBy("above").
        pivot("is_match", Seq("true","false")).
        count()
}

defined function crossTabs

Cancelled: Attempted to create case class to convert the pivot table array with strongly typed objects to help with the calculation of performance metrics. However, the column titles are true and false whcih are illegal column names.

org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class ResultArray(
    above:Boolean,
    true:Long,
    false:Long
)

//Create an encoder to use in converting the dataframe to a dataset.
val encoder = org.apache.spark.sql.Encoders.product[ResultArray]


In [27]:
val rawCT = crossTabs(scored,4.0)

rawCT: DataFrame = [above: boolean, true: bigint ... 1 more field]

In [28]:
rawCT.printSchema

root
 |-- above: boolean (nullable = false)
 |-- true: long (nullable = true)
 |-- false: long (nullable = true)



val ct = rawCT.as[encoder]

In [29]:
val ct=rawCT

ct: DataFrame = [above: boolean, true: bigint ... 1 more field]

In [30]:
ct.show()

show at cmd29.sc:1

10 / 10

show at cmd29.sc:1

200 / 200

show at cmd29.sc:1

...

show at cmd29.sc:1

100 / 100

show at cmd29.sc:1

75 / 75

+-----+-----+-------+
|above| true|  false|
+-----+-----+-------+
| true|20871|    637|
|false|   60|5727564|
+-----+-----+-------+



In [31]:
ct.cache()

res30: DataFrame = [above: boolean, true: bigint ... 1 more field]

### Performance Metrics

In [32]:
val results = ct.collect()

collect at cmd31.sc:1

10 / 10

collect at cmd31.sc:1

200 / 200

collect at cmd31.sc:1

200 / 200

results: Array[Row] = Array([true,20871,637], [false,60,5727564])

In [33]:
/**
Return the basic metrics for performance measurement
@param ct : Array[(above:bool,trues,falses)]
returns tuple (prec,recall,accuracy)
**/

def precRecall(ct: Array[Row]) ={
    val tp = (results(0)(1)).asInstanceOf[Long].toDouble
    val fp = (results(0)(2)).asInstanceOf[Long].toDouble
    val fn = (results(1)(1)).asInstanceOf[Long].toDouble
    val tn = (results(1)(2)).asInstanceOf[Long].toDouble
    
    val prec= tp/(tp+fp)
    val recall = tp/(tp+fn)
    val accuracy = (tp+tn)/(tp+fp+tn+fn)
    (prec,recall,accuracy)
}

defined function precRecall

In [34]:
precRecall(results)

res33: (Double, Double, Double) = (
  0.9703831132601822,
  0.9971334384405905,
  0.9998787643073772
)

In [35]:
case class performanceRow(
        level:Double,
        precision:Double,
        recall: Double,
        accuracy: Double
)

defined class performanceRow

In [36]:
def measure(level: Double, scored: DataFrame):performanceRow ={
    var ct = crossTabs(scored, level)
    var results = ct.collect()
    var perf = precRecall(results)
    return performanceRow(level, perf._1,perf._2,perf._3)
}

defined function measure

In [37]:
val levels = List(1.0,2.0,3.0,4.0,5.0)
levels.map(level=>measure(level,scored)).foreach(println)

collect at cmd35.sc:3

10 / 10

collect at cmd35.sc:3

200 / 200

collect at cmd35.sc:3

...

200 / 200

collect at cmd35.sc:3

200 / 200

performanceRow(1.0,0.9703831132601822,0.9971334384405905,0.9998787643073772)
performanceRow(2.0,0.9703831132601822,0.9971334384405905,0.9998787643073772)
performanceRow(3.0,0.9703831132601822,0.9971334384405905,0.9998787643073772)
performanceRow(4.0,0.9703831132601822,0.9971334384405905,0.9998787643073772)
performanceRow(5.0,0.9703831132601822,0.9971334384405905,0.9998787643073772)


levels: List[Double] = List(1.0, 2.0, 3.0, 4.0, 5.0)

In [0]:
measure(0.0,scored)

cmd0.sc:1: not found: value measure
val res0 = measure(0.0,scored)
           ^cmd0.sc:1: not found: value scored
val res0 = measure(0.0,scored)
                       ^Compilation Failed

: 

In [39]:
crossTabs(scored,0.0).show()

show at cmd38.sc:1

10 / 10

show at cmd38.sc:1

200 / 200

show at cmd38.sc:1

...

show at cmd38.sc:1

100 / 100

show at cmd38.sc:1

75 / 75

+-----+-----+-------+
|above| true|  false|
+-----+-----+-------+
| true|20931|5728201|
+-----+-----+-------+



In [0]:
for (i<- 1 to 5){
    crossTabs(scored,i).show()
}


cmd0.sc:2: not found: value crossTabs
    crossTabs(scored,i).show()
    ^cmd0.sc:2: not found: value scored
    crossTabs(scored,i).show()
              ^Compilation Failed

: 